In [3]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
!pip install urduhack
#!pip install transformers-phobert

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 2.3.1 requires numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.1 which is incompatible.



  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Attempting uninstall: keras-preprocessing
    Found existing installation: Keras-Preprocessing 1.1.0
    Uninstalling Keras-Preprocessing-1.1.0:
      Successfully uninstalled Keras-Preprocessing-1.1.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.0.0
    Uninstalling tensorflow-estimator-2.0.0:
      Successfully uninstalled tensorflow-estimator-2.0.0
  Attempting uninstall: gast
    Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.0.0

^C


In [4]:
import tensorflow as tf
from urduhack import normalization
from urduhack.preprocessing import remove_punctuation, normalize_whitespace
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input, Dropout
from tensorflow.keras.layers import Concatenate, Conv1D, MaxPool1D, LSTM, BatchNormalization
from transformers import XLMRobertaTokenizer, TFXLMRobertaModel

In [5]:
FILTER_SIZES = [3, 4]
NUM_FILTERS = 4
MAX_SEQUENCE_LENGTH = 512

In [9]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content' 
!kaggle datasets download -d akkefa/imdb-dataset-of-50k-movie-translated-urdu-reviews
!unzip \*.zip && rm *.zip

 46% 15.0M/32.3M [00:00<00:00, 58.5MB/s]
100% 32.3M/32.3M [00:00<00:00, 107MB/s] 


In [6]:
#imdb_df = pd.read_csv("/content/imdb_urdu_reviews_train.csv")
imdb_df = pd.read_csv("C:\\Sandbox\\GitHub\\data\\imdb_urdu_reviews_train.csv")

In [7]:
imdb_df.head()

,review,sentiment
0,میں نے اسے 80 کی دہائی کے وسط میں ایک کیبل گائ...,positive
1,چونکہ میں نے 80 کی دہائی میں انسپکٹر گیجٹ کارٹ...,negative
2,ایک ایسے معاشرے کی حالت کے بارے میں تعجب کرتا ...,positive
3,مفید البرٹ پیون کی طرف سے ایک اور ردی کی ٹوکری...,negative
4,یہ کولمبو ہے جس کی ہدایتکاری اپنے کیریئر کے اب...,positive


In [8]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     40000 non-null  object
 1   sentiment  40000 non-null  object
dtypes: object(2)
memory usage: 625.1+ KB


In [9]:
imdb_df.describe()

,review,sentiment
count,40000,40000
unique,39737,2
top,آج کا شو پسند آیا !!! یہ ایک قسم تھی اور نہ صر...,negative
freq,4,20082


In [10]:
imdb_df.columns

Index(['review', 'sentiment'], dtype='object')

In [ ]:
plt.hist(imdb_df["sentiment"]);

In [ ]:
imdb_df["review"][0]

In [ ]:
# Extract out sentences and labels
sentences = imdb_df['review'].tolist()
labels = dataset['sentiment'].tolist()

# Print some example sentences and labels
for x in range(2):
  print(sentences[x])
  print(labels[x])
  print("\n")

In [20]:
# Removing punctuations and unnecessary whitespaces from the text
imdb_df["review"] = imdb_df["review"].apply(remove_punctuation)
imdb_df["review"] = imdb_df["review"].apply(normalize_whitespace)
#imdb_df["review"] = imdb_df["review"].apply(normalize)

NameError: ignored

In [21]:
#clean and normalized text
imdb_df["review"][0]

'میں نے اسے 80 کی دہائی کے وسط میں ایک کیبل گائیڈ اسکائینجر ہنٹ پہلو نے مجھ سے اپیل کی تھی کی علامت پر مبنی ایک نوعمر کی حیثیت سے ٹیپ کیا اس فلم کی کوئی معلومات یا توقعات نہیں تھیں کتنی خوشگوار حیرت ہے جب میں نے اسے دیکھا یہ ایک ایسی تفریحی فلم تھی اور مجھے یاد ہے کہ اسے بار بار دیکھا جاتا تھا میں نے سوچا کہ یہ تصور اچھی طرح سے نافذ کیا گیا ہے میں نے مختلف گروہوں کے مابین بے ضرر مسابقت کا لطف اٹھایا اور مجھے لگتا ہے کہ مقتول کا شکار خود ہی کافی ہوشیار تھا کبھی کبھی ایسا لگتا ہے کہ لوگوں کو فلموں سے بہت زیادہ توقعات وابستہ ہیں تمام فلموں میں ایک بھاری پیغام یا شاندار اداکاری پروڈکشن کی قدریں یا خصوصی اثرات مرتب نہیں ہوں گے بعض اوقات فلمیں صرف تفریح \u200b\u200bاور تفریح \u200b\u200bکے لئے ہوتی ہیں اور یہ دونوں سطحوں پر کامیاب ہوتی ہے جڑواں اداکار اداکاروں کے تبصرے پڑھ کر بہت اچھا لگا میں نے یہ فلم برسوں میں نہیں دیکھی لیکن اگر میں یہ سمجھتا تھا کہ میں اس کے بارے میں اتنا ہی گرم اور جوش و خروش سے اپنا ردعمل ظاہر کروں گا جیسے میں نے نوعمری کی طرح کیا تھا یہاں تک کہ جب میں یہ ٹائپ کرتا ہوں

In [22]:
tokenizer = XLMRobertaTokenizer.from_pretrained("jplu/tf-xlm-roberta-base")
xlm_roberta = TFXLMRobertaModel.from_pretrained("jplu/tf-xlm-roberta-base")
xlm_roberta.trainable = False

Some weights of the model checkpoint at jplu/tf-xlm-roberta-base were not used when initializing TFXLMRobertaModel: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaModel were initialized from the model checkpoint at jplu/tf-xlm-roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


## Vector Visualization

In [23]:
tokens = tokenizer.tokenize(imdb_df["review"][0])
tokens

['▁میں',
 '▁نے',
 '▁اسے',
 '▁80',
 '▁کی',
 '▁دہائی',
 '▁کے',
 '▁وسط',
 '▁میں',
 '▁ایک',
 '▁کی',
 'بل',
 '▁گ',
 'ائیڈ',
 '▁اسکا',
 'ئی',
 'نجر',
 '▁ہ',
 'نٹ',
 '▁پہلو',
 '▁نے',
 '▁مجھ',
 '▁سے',
 '▁اپیل',
 '▁کی',
 '▁تھی',
 '▁کی',
 '▁علامت',
 '▁پر',
 '▁مبنی',
 '▁ایک',
 '▁نوع',
 'مر',
 '▁کی',
 '▁حیثیت',
 '▁سے',
 '▁ٹی',
 'پ',
 '▁کیا',
 '▁اس',
 '▁فلم',
 '▁کی',
 '▁کوئی',
 '▁معلومات',
 '▁یا',
 '▁توقع',
 'ات',
 '▁نہیں',
 '▁تھیں',
 '▁کتنی',
 '▁خوشگوار',
 '▁حیرت',
 '▁ہے',
 '▁جب',
 '▁میں',
 '▁نے',
 '▁اسے',
 '▁دیکھا',
 '▁یہ',
 '▁ایک',
 '▁ایسی',
 '▁تفریح',
 'ی',
 '▁فلم',
 '▁تھی',
 '▁اور',
 '▁مجھے',
 '▁یاد',
 '▁ہے',
 '▁کہ',
 '▁اسے',
 '▁بار',
 '▁بار',
 '▁دیکھا',
 '▁جاتا',
 '▁تھا',
 '▁میں',
 '▁نے',
 '▁سوچ',
 'ا',
 '▁کہ',
 '▁یہ',
 '▁تصور',
 '▁اچھی',
 '▁طرح',
 '▁سے',
 '▁نافذ',
 '▁کیا',
 '▁گیا',
 '▁ہے',
 '▁میں',
 '▁نے',
 '▁مختلف',
 '▁گروہ',
 'وں',
 '▁کے',
 '▁ما',
 'بین',
 '▁بے',
 '▁ضرر',
 '▁م',
 'سابق',
 'ت',
 '▁کا',
 '▁لطف',
 '▁اٹھایا',
 '▁اور',
 '▁مجھے',
 '▁لگتا',
 '▁ہے',
 '▁کہ',
 '▁مقتول',
 '▁کا',
 '▁ش

In [ ]:
token_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(imdb_df["review"][0]))
token_ids

In [25]:
#sentence output gives token level vector representation
#pooled output gives sentence level vector representation
sentence_output, pooled_output = xlm_roberta(np.array([token_ids]))

print(sentence_output.shape, pooled_output.shape)

(1, 314, 768) (1, 768)


In [27]:
for i in range(2):
    print(str(tokens[i]) + ": " + str(sentence_output[:, i, :]))

▁میں: tf.Tensor(
[[ 5.76357916e-02  7.77227730e-02  8.50180015e-02 -1.09151918e-02
   6.18211459e-04 -4.60526794e-02  2.30736025e-02 -2.82477364e-02
   5.60150519e-02 -1.19721994e-01  2.69100275e-02  9.61062983e-02
  -1.28265694e-02 -8.93310364e-03  4.52610590e-02  5.00317775e-02
  -4.50898521e-02 -5.20010591e-02  3.36608738e-02  4.53229882e-02
   4.07336280e-02 -5.49936108e-03  3.62981893e-02  4.02036309e-02
  -7.16611296e-02  3.41757461e-02 -3.71506959e-02  2.65742708e-02
   9.17291790e-02  3.79055142e-02  1.14243321e-01 -8.64227489e-03
   2.07649376e-02  6.34058751e-03  9.50742140e-02 -7.83666130e-03
  -1.04717471e-01 -6.11959249e-02  9.33857784e-02  9.35193300e-02
   6.11648485e-02  1.12425290e-01  1.67309772e-02 -5.65585233e-02
   4.72922921e-02 -7.08445311e-02 -8.56816098e-02 -4.30910289e-02
   1.01050008e-02 -9.94359106e-02  5.65509051e-02  4.09313999e-02
   6.46292418e-02  3.59194353e-02 -1.26892075e-01 -1.09059647e-01
   1.20491974e-01  2.30638310e-02  8.77624005e-03  1.081830

In [28]:
texts = imdb_df["review"].values.tolist()

In [29]:
#tokenizer.encode directly converts text to tokens to ids and adds special_tokens(cls_tokens, unk_token, sep_tokens) as well as padding
encoded = []
for text in texts:
    input_ids = tokenizer.encode(text, max_length=MAX_SEQUENCE_LENGTH, pad_to_max_length=True, add_special_tokens=True, truncation=True)
    encoded.append(input_ids)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [30]:
data = np.array(encoded)

In [31]:
labels = imdb_df["sentiment"].values.tolist()
from sklearn.preprocessing import LabelEncoder
en = LabelEncoder()
labels = en.fit_transform(labels)
labels = tf.keras.utils.to_categorical(labels)

In [33]:
n_classes = len(np.unique(labels))

In [34]:
VALIDATION_SPLIT = 0.15
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [35]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
sequence_ouput, _ = xlm_roberta(sequence_input)

conv_blocks = []
for k_size in FILTER_SIZES:
    conv = Conv1D(filters=NUM_FILTERS,
                         kernel_size=k_size,
                         padding="valid",
                         activation="relu",
                         strides=1)(sequence_ouput)
    conv = Dropout(0.25)(conv)
    conv = MaxPooling1D(pool_size=MAX_SEQUENCE_LENGTH - k_size + 1)(conv)
    conv = Flatten()(conv)
    conv_blocks.append(conv)
concat = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
x = Dropout(0.2)(concat)
x = Dense(512, activation="relu")(x)
preds = Dense(n_classes, activation="softmax")(x)

In [36]:
model = tf.keras.Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
              metrics=['acc'])

In [37]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
tfxlm_roberta_model (TFXLMRober ((None, 512, 768), ( 278043648   input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 510, 4)       9220        tfxlm_roberta_model[0][0]        
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 509, 4)       12292       tfxlm_roberta_model[0][0]        
_______________________________________________________________________________________

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, mode='auto')
history = model.fit(x_train, y_train, validation_data=(x_val, y_val),
                    epochs=10, batch_size=16, callbacks=[es])

Epoch 1/10
2125/2125 [==============================] - 1772s 834ms/step - loss: 0.6942 - acc: 0.5014 - val_loss: 0.6928 - val_acc: 0.4983
Epoch 2/10
2125/2125 [==============================] - 1781s 838ms/step - loss: 0.6922 - acc: 0.5145 - val_loss: 0.6931 - val_acc: 0.4985
Epoch 3/10
 167/2125 [=>............................] - ETA: 23:46 - loss: 0.6912 - acc: 0.5333

In [ ]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show();
    
plot_graphs(history, "acc");
plot_graphs(history, "loss");

In [ ]:
text = "اگر آپ ہیل بوائے کے ساتھ کھوئے ہوئے صندوق کے چھاپوں کو عبور کرتے ہیں اور پرل ہاربر اور ہدایت کار جو جانسٹن کے اپنے دی روکیٹیئر کو شامل کرتے ہیں تو ، آپ اس فلم کی تخلیق کرنے کے راستے میں ٹھیک ہوجائیں گے۔ یہ اچھی طرح سے چل رہا ہے ، محب وطن جنگ عظیم دوم کی مہم جوئی سائنس فائی / فنتاسی تصوف کا ایک کم جمہوریہ اور ہیرو ہے جس کی جڑیں اکٹھا کرنا بہت آسان ہے۔ راجرز / کیپٹن امریکہ ، بہت اچھا آدمی ہے ، حقیقت میں ، کہ وہ کبھی کبھی تھوڑا سا کمزور ہوتا ہے - وہ یقینا a ایک بہت بڑا رول ماڈل ہے ، لیکن آپ اپنے آپ کو یہ خواہش پاسکتے ہیں کہ اس کے پاس تھوڑا سا ولورائن یا ٹونی اسٹارک کی سنیارک تھی۔ (فلم میں مجموعی طور پر مزاح سے تھوڑا سا مختصر ہے ، دراصل ، کیپٹن امریکہ کی رگ ٹیگ آف سپاہیوں کی ٹیم زیادہ تر قہقہوں دیتی ہے ، لیکن ان میں اتنا زیادہ سکرین ٹائم نہیں ملتا ہے۔)"

In [ ]:
text = remove_punctuation(text)
text = normalize_whitespace(text)
text = normalize(text)

In [ ]:
encoded = tokenizer.encode(text, add_special_tokens=True, max_length=MAX_SEQUENCE_LENGTH, pad_to_max_length=True)
encoded = np.asarray([encoded])
predictions = model.predict(encoded)

In [ ]:
print(np.argmax(predictions))